# ModernAI Pro: From Youtube to Audio summaries

---





In this notebook let's use Whisper and ChatGPT to summarize a Youtube video

## 1.Downloading Youtube videos
Note: The legality of this depends on the geography

In [ ]:
!pip install pytube -q
from pytube import YouTube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00


In [ ]:
url = "https://www.youtube.com/watch?v=zAjJYkUnTEs" #short
#url = "https://www.youtube.com/watch?v=3x4zM7O8pZI" #long
yt = YouTube(url)
audio = yt.streams.filter(only_audio=True).first().download()

In [ ]:
#Test if the Audio plays before we use whisper
from IPython.display import Audio
Audio(audio, rate = 22050)

## 2. Trancribe Audio with Whisper

In [ ]:
#First we will setup whisper
! pip install git+https://github.com/openai/whisper.git -q
import whisper
model = whisper.load_model("medium")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.2 MB/s eta 0:00:00


100%|█████████████████████████████████████| 1.42G/1.42G [01:08<00:00, 22.3MiB/s]


In [ ]:
transcript = model.transcribe(audio,fp16=False)['text']

In [ ]:
transcript

## 3. Clean the sentences and summarize with LLMs

In [ ]:
from google.colab import userdata

!pip install git+https://gitlab.com/gauthammsam/mitrallm.git --quiet
from mitrallm import MitraLLM
llm = MitraLLM(
    token     = userdata.get("MITRA_TOKEN"),
    accessapi = userdata.get("MITRA_ENDPOINT")
)

MitraLLM
Params: {'URL': 'http://35.170.200.80:8000'}


In [ ]:
print(llm.invoke("Can you summarize and provide context to this audio. Also guess what show this is from and the scene: "+transcript))

In [ ]:
response

'This audio is from the critically acclaimed American television series "Breaking Bad." The scene is from the episode "Cornered," which is the sixth episode of the fourth season. The two characters speaking are Walter White, played by Bryan Cranston, and his wife, Skyler White, played by Anna Gunn.\n\nIn this intense scene, Skyler is confronting Walt about the dangers of his involvement in the methamphetamine business. She is suggesting they go to the police to protect themselves and their family from harm. Skyler argues that the truth—Walt being a cancer-stricken high school chemistry teacher turned meth producer out of desperation—would garner sympathy and be their best out.\n\nWalt, however, resists the idea of going to the police and rejects the notion that he is in danger or a victim of circumstance. Instead, he asserts his power and control within the drug trade by telling Skyler that he is not the one at risk, but rather the one who poses a threat to others. He states that he is

## Putting it all together in one function

In [ ]:
def summarize_yt(url, llm):
  yt = YouTube(url)
  audio = yt.streams.filter(only_audio=True).first().download()
  print("Audio downloaded")

  transcript = model.transcribe(audio,fp16=False)['text']
  print(transcript)

  result = llm.invoke("Summarize this Youtube video transcript and provide context if possible: "+transcript)

  return result

In [ ]:
text = summarize_yt("https://www.youtube.com/watch?v=zAjJYkUnTEs", llm)
text

Audio downloaded
 I've said it before if you are in danger we go to the police. I don't want to hear about the police. I do not say that lightly. I know what it could do to this family but if it's the only real choice we have, if it's either that or you getting shot when you open your front door. I don't want to hear about the police. You are not some hardened criminal Walt. You are in over your head. That's what we tell them. That's the truth. That is not the truth. Of course it is. A school teacher, cancer, desperate for money. Okay we're done here. Roped into working for unable to even quit. You told me that yourself Walt. Jesus what was I thinking? Walt please let's both of us stop trying to justify this whole thing and admit you're in danger. Who are you talking to right now? Who is it you think you see? Do you know how much I make a year? I mean even if I told you you wouldn't believe it. You know what would happen if I suddenly decided to stop going into work? A business big eno

'The transcript you\'ve provided appears to be a dialogue from a scene in the TV series Breaking Bad, specifically a confrontation between the main character Walter White and his wife, Skyler White. The show follows Walter White, a high school chemistry teacher turned methamphetamine manufacturing drug dealer, after he is diagnosed with terminal lung cancer.\n\nIn this scene, Skyler is expressing her concerns for the safety of their family, urging Walter to go to the police and confess, given the dangerous situation they are in because of Walter\'s involvement in drug trafficking. She sees going to the authorities as a potential solution to protect him from being harmed or killed by rival criminals.\n\nWalter\'s response to Skyler\'s suggestion is dismissive and defiant. He emphatically rejects her idea and proceeds to assert his control over the situation. He hints at the scale and success of his drug operation, suggesting that it\'s so significant that its collapse would have noticea